<div class="alert alert-block alert-info">
<h1><b>Naloga: </b></h1>
Ustvarite skripto, ki nam pove v kateri fazi COVID semaforja se nahajamo glede na <b>Št. pozitivnih testov - 7dnevno povprečje</b> in <b>število hospitaliziranih</b>.
    
Podatke pridobite na https://api.sledilnik.org/api/stats. Podani so v JSON formatu, z najnovejšimi podatki na koncu.
</div>


In [7]:
# Rešitev
import requests

def determine_zone(seven_day_average, hospitalised):
    txt = f"with {round(seven_day_average)} cases on average per day in the last week, and {hospitalised} hospitalised, we are in the "
    if seven_day_average < 300 and hospitalised == 0:
        txt = txt + "green zone"
    elif seven_day_average < 600 and hospitalised < 500:
        txt = txt + "yellow zone"
    elif seven_day_average < 1000 and hospitalised < 1000:
        txt = txt + "orange zone"
    elif seven_day_average < 1350 and hospitalised < 1200:
        txt = txt + "red zone"
    else:
        txt = txt + "black zone"
    return txt

covid_stats_url = "https://api.sledilnik.org/api/stats"
r = requests.get(covid_stats_url)
covid_stats = r.json()

In [21]:
seven_day_covid = covid_stats[-7:]
covid_tests = [day['tests']['positive']['today'] for day in seven_day_covid]
seven_day_average = sum(covid_tests) / len(covid_tests)
hospitalised = covid_stats[-1]['statePerTreatment']['inHospital']

determine_zone(seven_day_average, hospitalised)

'with 706 cases on average per day in the last week, and 629 hospitalised, we are in the orange zone'

<div class="alert alert-block alert-info">
<h1><b>Naloga: </b></h1>
Ustvarite skripto, ki nam pridobi podatke slovenski občin na spletni strani: <a href=https://skupnostobcin.si/podatki/zemljevid-obcin-tabela/#p2>https://skupnostobcin.si/podatki/zemljevid-obcin-tabela/#p2</a>
    
Skripta naj ustvari nov dictionary v katerega shrani Ime občine, število prebivalcev, površino (km2) in število prebivalca/km2 - ni potrebno upoštevati decimalk.
    
    data {
        "Ljubljana": {
            "Št. prebivalcev": 273091,
            "Površina (km2)": 275,
            "Št.preb/km2": 993
            },
        "Piran": {
            "Št. prebivalcev": 17268,
            "Površina (km2)": 44.6,
            "Št.preb/km2": 387
            },
        ...
        }
    
</div>


In [12]:
# Rešitev
import pandas as pd

municipalities = pd.read_html("https://skupnostobcin.si/podatki/zemljevid-obcin-tabela/#p2")[0]
data = {}
for idx, municipality in municipalities.iterrows():
    name = municipality["Občina"]
    inhabitants = municipality["Št. prebivalcev"]
    area = municipality["Površina (km2)"]
    data[name] = {"St. prebivalcev": inhabitants,
                  "Površina (km2)": area,
                  "St.preb/km2": round(inhabitants/area),
                }

{'St. prebivalcev': 273091, 'Površina (km2)': 275.0, 'St.preb/km2': 993}

<div class="alert alert-block alert-info">
<h1><b>Naloga: </b></h1>
Ustvarite skripto, ki pridobi informacije o 250 najbolje ocenjenih filmih.
    
https://www.imdb.com/chart/top/?ref_=nv_mv_250
    
Skripta naj pridobi naslov filma, oceno filma in trajanje filma. Trajanje filma dobite, če odprete specifični film.
</div>

```python
Output:
Kaznilnica odrešitve
9.2
2h 22min

Boter
9.1
2h 55min

Boter, II. del
9.0
3h 22min

Vitez teme
9.0
2h 32min

...
```

In [1]:
# Rešitev
import requests
import json
from bs4 import BeautifulSoup
import parse
imdb_url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
r = requests.get(imdb_url)
soup = BeautifulSoup(r.content, "html.parser")
imdb_table = soup.find_all("table")[0]

In [24]:
def get_movie_duration(movie_url):
    movie_request = requests.get(movie_url)
    movie_soup = BeautifulSoup(movie_request.content, "html.parser")
    movie = json.loads("".join(movie_soup.find("script", {"type":"application/ld+json"}).contents))
    
    duration_format = "PT{hours:n}H{minutes:n}M"
    duration = parse.parse(duration_format, movie["duration"])
    
    return duration

In [30]:
create_url = lambda href_part : f"http://www.imdb.com{href_part}"

rows = imdb_table.find_all("tr")
for row in rows[:]:
    full_title = row.find("td", class_="titleColumn")
    rating = row.find("td", class_="ratingColumn")
    if not full_title: 
        continue
    
    title = full_title.find("a")
    movie_url = create_url(title['href'])
    duration = get_movie_duration(movie_url)
    print(title.text.strip(), 
          rating.text.strip(), 
          f"{duration['hours']}h {duration['minutes']}min",
          sep="\n", end="\n\n")
    


Kaznilnica odrešitve
9.2
2h 22min

Boter
9.1
2h 55min

Boter, II. del
9.0
3h 22min

Vitez teme
9.0
2h 32min

12 jeznih mož
8.9
1h 36min

Schindlerjev seznam
8.9
3h 15min

Gospodar prstanov: Kraljeva vrnitev
8.9
3h 21min

Šund
8.8
2h 34min

Dober, grd, hudoben
8.8
2h 58min

Gospodar prstanov: Bratovscina prstana
8.8
2h 58min

Klub golih pesti
8.8
2h 19min

Forrest Gump
8.8
2h 22min

Izvor
8.7
2h 28min

Gospodar prstanov: stolpa
8.7
2h 59min

Star Wars: Episode V - The Empire Strikes Back
8.7
2h 4min

Matrica
8.6
2h 16min

Dobri fantje
8.6
2h 26min

Let nad kukavičjim gnezdom
8.6
2h 13min

Sedem samurajev
8.6
3h 27min

Sedem
8.6
2h 7min

Življenje je lepo
8.6
1h 56min

Cidade de Deus
8.6
2h 10min

Ko jagenjčki obmolknejo
8.6
1h 58min

Cudovito zivljenje
8.6
2h 10min

Reševanje vojaka Ryana
8.6
2h 49min

Vojna zvezd: Epizoda IV - Novo upanje
8.6
2h 1min

Zelena milja
8.5
3h 9min

Čudežno potovanje
8.5
2h 5min

Medzvezdje
8.5
2h 49min

Parazit
8.5
2h 12min

Léon
8.5
1h 50min

Seppuku
8.5
2

TypeError: 'NoneType' object is not subscriptable

2